## This notebook is a modified version of [the notebook for fine-tuning Llama 3.1 8B by Unsloth](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing).

### Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

auto_delete_colab_session = False
generate_dataset_from_scratch = False
unsloth_model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"
output_hf_model_name = "author/Meta-Llama-3.1-8B-Instruct-clin-es"

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = unsloth_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('HF_TOKEN') # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
dataset_path = "/content/drive/MyDrive/AI/MultiClinSum 2025/Data"

#### Run once to prepare the dataset

In [ ]:
import os
import pandas as pd
from datasets import Dataset

if generate_dataset_from_scratch:
  full_texts_path = "/content/drive/MyDrive/AI/MultiClinSum 2025/Data/fulltext"
  summaries_path = "/content/drive/MyDrive/AI/MultiClinSum 2025/Data/summaries"

  full_texts = []

  full_text_file_names_sorted = sorted(
      [f for f in os.listdir(full_texts_path) if f.endswith(".txt")],
      key=lambda x: int(x.split('_')[-1].removesuffix('.txt'))
  )

  for file_name in full_text_file_names_sorted:
    print(file_name)
    with open(f"{full_texts_path}/{file_name}", "r") as in_file:
      full_texts.append(in_file.read())

  summaries = []

  summaries_file_names_sorted = sorted(
      [f for f in os.listdir(summaries_path) if f.endswith(".txt")],
      key=lambda x: int(x.split('_')[-2])
  )

  for file_name in summaries_file_names_sorted:
    print(file_name)
    with open(f"{summaries_path}/{file_name}", "r") as in_file:
      summaries.append(in_file.read())

  dataset_df = pd.DataFrame({'full_text': full_texts, 'summaries': summaries})
  dataset = Dataset.from_pandas(dataset_df).train_test_split(test_size=92)
  dataset.save_to_disk(dataset_path)

#### Load dataset and apply chat template

In [ ]:
from datasets import load_from_disk

train_dataset = load_from_disk(dataset_path+"/train")
test_dataset = load_from_disk(dataset_path+"/test")

print(train_dataset)
print(test_dataset)

In [ ]:
def format_chat_messages(example):
  text_to_summarize = example["full_text"]
  summary = example["summaries"] # Fix the name pls
  messages =  [
  {
    "role": "system",
    "content": "Eres un asistente médico experto en redactar resúmenes clínicos concisos y precisos en español. Tu tarea es analizar informes de casos clínicos y generar un resumen narrativo que destaque los aspectos más relevantes para profesionales de la salud, incluyendo hallazgos médicos clave como resultados de pruebas importantes y otras observaciones significativas. El resumen debe presentarse en un solo párrafo, sin utilizar viñetas, encabezados ni ningún tipo de formato especial."
  },
  {
    "role": "user",
    "content": f"Por favor, resume el siguiente informe de caso clínico en español, enfocándote en:\n\n- Datos clínicos clave\n- Resultados de pruebas importantes (laboratorio, imagenología, etc.)\n- Diagnóstico\n- Tratamiento\n- Evolución del paciente\n\nAsegúrate de que el resumen sea claro, coherente y útil para profesionales médicos. Presenta el resumen en un solo párrafo, sin utilizar viñetas, encabezados ni ningún tipo de formato especial.\n\n{text_to_summarize}"
  },
  {
    "role": "assistant",
    "content": summary
  }
  ]

  fomatted_text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
  return { "text" : fomatted_text }


In [ ]:
dataset = train_dataset.map(format_chat_messages).remove_columns(['full_text', 'summaries'])

In [ ]:
dataset[0]

<a name="Train"></a>
### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 25,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.push_to_hub(output_hf_model_name, token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub(output_hf_model_name, token = userdata.get('HF_TOKEN')) # Online saving

In [ ]:
model.push_to_hub_merged(output_hf_model_name + "-merged-16bit", tokenizer, save_method = "merged_16bit", token = userdata.get('HF_TOKEN'))

In [ ]:
if auto_delete_colab_session:
  from google.colab import runtime
  runtime.unassign()